### Attribute Information

- **time:** Time to failure or censoring
- **trt:** Treatment indicator 
  - 0 = ZDV only
  - 1 = ZDV + ddI
  - 2 = ZDV + Zal
  - 3 = ddI only
- **age:** Age (years) at baseline
- **wtkg:** Weight (kg) at baseline
- **hemo:** Hemophilia 
  - 0 = no
  - 1 = yes
- **homo:** Homosexual activity 
  - 0 = no
  - 1 = yes
- **drugs:** History of IV drug use 
  - 0 = no
  - 1 = yes
- **karnof:** Karnofsky score (on a scale of 0-100)
- **oprior:** Non-ZDV antiretroviral therapy pre-175 
  - 0 = no
  - 1 = yes
- **z30:** ZDV in the 30 days prior to 175 
  - 0 = no
  - 1 = yes
- **preanti:** Days pre-175 anti-retroviral therapy
- **race:** Race 
  - 0 = White
  - 1 = non-white
- **gender:** Gender 
  - 0 = Female
  - 1 = Male
- **str2:** Antiretroviral history 
  - 0 = naive
  - 1 = experienced
- **strat:** Antiretroviral history stratification 
  - 1 = 'Antiretroviral Naive'
  - 2 = '> 1 but <= 52 weeks of prior antiretroviral therapy'
  - 3 = '> 52 weeks'
- **symptom:** Symptomatic indicator 
  - 0 = asymptomatic
  - 1 = symptomatic
- **treat:** Treatment indicator 
  - 0 = ZDV only
  - 1 = others
- **offtrt:** Indicator of off-treatment before 96 ± 5 weeks 
  - 0 = no
  - 1 = yes
- **cd40:** CD4 count at baseline
- **cd420:** CD4 count at 20 ± 5 weeks
- **cd80:** CD8 count at baseline
- **cd820:** CD8 count at 20 ± 5 weeks
- **infected:** Is infected with AIDS 
  - 0 = No
  - 1 = Yes


In [1]:
import pandas as pd
import numpy as np

In [4]:
aids = pd.read_csv('data/AIDS_Classification_50000.csv')

In [5]:
aids.head()

,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820,infected
0,1073,1,37,79.46339,0,1,0,100,0,1,...,1,2,0,1,0,322,469,882,754,1
1,324,0,33,73.02314,0,1,0,90,0,1,...,1,3,1,1,1,168,575,1035,1525,1
2,495,1,43,69.47793,0,1,0,100,0,1,...,1,1,0,0,0,377,333,1147,1088,1
3,1201,3,42,89.15934,0,1,0,100,1,1,...,1,3,0,0,0,238,324,775,1019,1
4,934,0,37,137.46581,0,1,0,100,0,0,...,0,3,0,0,1,500,443,1601,849,0


In [9]:
aids.shape

(50000, 23)

In [7]:
type(aids)

pandas.core.frame.DataFrame